__Importing Pandas__

In [1]:
import pandas as pd

__Creating a function to get details for every city:__

* __API 1__ | Typethread for location id<br/>
 *location and location id needed for restaurant details*<br/>
<br/>
* __API 2__ | Restaurant details in the location<br/>
 *restaurant details in the location (Top 50 available at max)*<br/>
<br/>
* __Data collection in a data frame__

In [2]:
def restodata(name):
    import requests
    
    ##############################
    # Typethread for location id # 
    ##############################

    url = "https://worldwide-restaurants.p.rapidapi.com/typeahead"
    
    payload = "q={}&language=en_US".format(name)
    headers = {
        "content-type": "application/x-www-form-urlencoded",
        "X-RapidAPI-Host": "worldwide-restaurants.p.rapidapi.com",
        "X-RapidAPI-Key": "4b850d85a8msh680bf8cc3e0ace9p1b4215jsncce8c62767c5"              #APIKey
    }

    response1 = requests.request("POST", url, data=payload, headers=headers).json()
    L=response1['results']['data'][0]['result_object']['location_id']
    loc_id = int(L)
    #return print(response1['results']['data'][0]['result_object']['name'],response1['results']['data'][0]['result_object']['location_id'])
    # got access to location and location id
    
    ######################################
    # Restaurant details in the location #
    ######################################
    url = "https://worldwide-restaurants.p.rapidapi.com/search"

    payload = "language=en_US&limit=100&location_id={}&currency=USD".format(loc_id)
    headers = {
        "content-type": "application/x-www-form-urlencoded",
        "X-RapidAPI-Host": "worldwide-restaurants.p.rapidapi.com",
        "X-RapidAPI-Key": "4b850d85a8msh680bf8cc3e0ace9p1b4215jsncce8c62767c5"               #APIKey
    }

    response = requests.request("POST", url, data=payload, headers=headers).json()
    #return(response)
    # got restaurant details
    
    ###################################
    # Data collection in a data frame #
    ###################################
    
    restaurantdata=[]
    for i in range(0,len(response['results']['data'])):
        data={'name': response['results']['data'][i]['name'],
              'latitude': response['results']['data'][i]['latitude'],
              'longitude': response['results']['data'][i]['longitude'],
              'location_id': response['results']['data'][i]['location_id'],
              'num_reviews': response['results']['data'][i]['num_reviews'],
              'ranking_position': response['results']['data'][i]['ranking_position'],
              'ranking_denominator': response['results']['data'][i]['ranking_denominator'],
              'ranking_category': response['results']['data'][i]['ranking_category'],
              'rating': response['results']['data'][i]['rating'],
              'address': response['results']['data'][i]['address'],
              'cuisine': response['results']['data'][i]['cuisine'][0]['name'],
              #'dietary_restrictions': response['results']['data'][i]['dietary_restrictions'][0]['name'],
              'establishment_types': response['results']['data'][i]['establishment_types'][0]['name'],
              'price': response['results']['data'][i].get('price') #Key Error handling
             }
        restaurantdata.append(data)
        #print((response['results']['data'][i]))
    restaurantdata_df = pd.DataFrame()
    restaurantdata_df = pd.DataFrame(restaurantdata)
    
    #Returning the response for data transformation later
    #Returning the main dataframe created
    
    return response,restaurantdata_df                                          # multiple return
  

In [24]:
response, resto_df = restodata('Karlsruhe')
resto_df.head(10)

,name,latitude,longitude,location_id,num_reviews,ranking_position,ranking_denominator,ranking_category,rating,address,cuisine,establishment_types,price
0,Restaurant Sonnenbad,49.012627,8.348418,5817948,213,1,604,restaurant,4.5,"Am Sonnenbad 1, 76189, Karlsruhe, Baden-Wurtte...",Mediterranean,Restaurants,$2 - $24
1,Pinsa Si,49.00722,8.39475,16810634,83,2,604,restaurant,5.0,"Karlstrasse 31, 76133 Karlsruhe, 76133, Karlsr...",Italian,Restaurants,None
2,Restaurant Terra Mare,49.003933,8.398199,7236591,423,3,604,restaurant,4.5,"Ritterstr. 19, 76137, Karlsruhe, Baden-Wurttem...",Seafood,Restaurants,$13 - $25
3,Toro Tapasbar,49.01077,8.393829,4054709,577,4,604,restaurant,4.5,"Akademiestrasse 57, 76133, Karlsruhe, Baden-Wu...",Seafood,Restaurants,None
4,Casa do Jose,49.00582,8.407025,3737103,337,5,604,restaurant,4.5,"Kriegsstr. 92, 76133, Karlsruhe, Baden-Wurttem...",Seafood,Restaurants,None
5,Alte Brauerei,49.053356,8.531175,5531196,226,6,604,restaurant,4.5,"Bruchsaler Str. 22, 76356 Weingarten, Karlsruh...",Barbecue,Restaurants,None
6,DeliBurgers,49.01103,8.395794,9734212,276,7,604,restaurant,4.5,"Akademiestr. 39 Next to Europaplatz, 76133, Ka...",American,Restaurants,$11 - $20
7,Sokrates,48.996128,8.391512,1040113,287,8,604,restaurant,4.5,"Welfenstr. 18, 76137, Karlsruhe, Baden-Wurttem...",Seafood,Restaurants,None
8,Il Teatro2,49.003975,8.40281,2073052,82,9,604,restaurant,5.0,"Ettlingerstrasse 2c, 76137, Karlsruhe, Baden-W...",Seafood,Restaurants,None
9,Osteria Carre,49.00929,8.37632,6214144,180,10,604,restaurant,4.5,"Nelkenstr. 19, 76135, Karlsruhe, Baden-Wurttem...",Italian,Restaurants,None


__Data Transformation__

* Checking the cuisine data for each popular city to add their columns in the dataframe

In [25]:
# response, resto_df = restodata('berlin')
# resto_df

#Checked the cuisine data for each popular city and made the necessary changes

# 1. Berlin
# Extracted Bar, Asian, Healthy, German, Central European, Mediterranean, European, Vegan Options, Gluten Free Options, Seafood  data for separate categories

# 2. Munich
# Added Cafe, Indian | Edited Bars (added Dining bars, Gastropub)

# 3. Hamburg
# Added Steakhouse

# 4. Frankfurt
# No changes required

# 5. Essen
# Added American, pub (edited in bar)

# 6. Stuttgart
# Added Japanese

# 7. Dortmund
# No changes required

# 8. Duesseldorf
# Added Italian

# 9. Hannover
# No changes required

# 10. Leipzig
# No changes required

# Further cities required no additional changes

cuisineslist=[]
for i in range(0,len(response['results']['data'])):
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        cuisineslist.append(response['results']['data'][i]['cuisine'][j]['name'])
cuisineslist
from collections import Counter
c1=Counter(cuisineslist).keys() # equals to list(set(words))
c2=Counter(cuisineslist).values() # counts the elements' frequency
print(Counter(cuisineslist))



Counter({'Vegetarian Friendly': 49, 'Vegan Options': 28, 'European': 25, 'Gluten Free Options': 21, 'Mediterranean': 15, 'Central European': 15, 'German': 13, 'Italian': 8, 'Seafood': 6, 'Bar': 6, 'Grill': 5, 'Asian': 5, 'Greek': 4, 'Diner': 4, 'Pizza': 3, 'Healthy': 3, 'American': 3, 'Cafe': 3, 'Spanish': 2, 'Barbecue': 2, 'Middle Eastern': 2, 'International': 2, 'Vietnamese': 2, 'Romana': 1, 'Lazio': 1, 'Central-Italian': 1, 'Contemporary': 1, 'Street Food': 1, 'Portuguese': 1, 'Chinese': 1, 'Thai': 1, 'African': 1, 'Ethiopian': 1, 'Lebanese': 1, 'Steakhouse': 1, 'Wine Bar': 1, 'Deli': 1, 'Sri Lankan': 1, 'Indian': 1, 'Balti': 1, 'Brew Pub': 1, 'Mexican': 1, 'Latin': 1, 'Central American': 1, 'South American': 1, 'Halal': 1, 'Japanese': 1, 'Sushi': 1})


* Making relevant categories from the available data <br/>
 *Adding the columns of each popular cuisine after checking for each city.*

In [26]:
Bar=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Bar' or response['results']['data'][i]['cuisine'][j]['name'] == 'Brew Pub' or response['results']['data'][i]['cuisine'][j]['name'] == 'Wine Bar' or response['results']['data'][i]['cuisine'][j]['name'] == 'Gastropub' or response['results']['data'][i]['cuisine'][j]['name'] == 'Dining bars' or response['results']['data'][i]['cuisine'][j]['name'] == 'Pub':    ## changes in parameter
            Bar.append(i)                                                    ## changes in list
print('Bar',Bar) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in Bar:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Bar']                                                          ## changes in column names
#df1
resto_df['Bar']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

Italian=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Italian' :    ## changes in parameter
            Italian.append(i)                                                    ## changes in list
print('Italian',Italian) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in Italian:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Italian']                                                          ## changes in column names
#df1
resto_df['Italian']=df1                                                          ## changes in column names
#resto_df
#----------------------------------------------------------------------------------------------------------------------------

Japanese=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Japanese' :    ## changes in parameter
            Japanese.append(i)                                                    ## changes in list
print('Japanese',Japanese) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in Japanese:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Japanese']                                                          ## changes in column names
#df1
resto_df['Japanese']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

American=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'American' :    ## changes in parameter
            American.append(i)                                                    ## changes in list
print('American',American) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in American:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['American']                                                          ## changes in column names
#df1
resto_df['American']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

Steakhouse=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Steakhouse' :    ## changes in parameter
            Steakhouse.append(i)                                                    ## changes in list
print('Steakhouse',Steakhouse) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in Steakhouse:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Steakhouse']                                                          ## changes in column names
#df1
resto_df['Steakhouse']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

Cafe=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Cafe' :    ## changes in parameter
            Cafe.append(i)                                                    ## changes in list
print('Cafe',Cafe) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in Cafe:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Cafe']                                                          ## changes in column names
#df1
resto_df['Cafe']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

Indian=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Indian' :    ## changes in parameter
            Indian.append(i)                                                    ## changes in list
print('Indian',Indian) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in Indian:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Indian']                                                          ## changes in column names
#df1
resto_df['Indian']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

Asian=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Asian':    ## changes in parameter
            Asian.append(i)                                                    ## changes in list
print('Asian',Asian) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in Asian:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Asian']                                                          ## changes in column names
#df1
resto_df['Asian']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

Healthy=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Healthy':    ## changes in parameter
            Healthy.append(i)                                                    ## changes in list
print('Healthy',Healthy) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in Healthy:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Healthy']                                                          ## changes in column names
#df1
resto_df['Healthy']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

ger=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'German':    ## changes in parameter
            ger.append(i)                                                    ## changes in list
print('German',ger) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()  
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in ger:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['German']                                                          ## changes in column names
#df1
resto_df['German']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

ce=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Central European':    ## changes in parameter
            ce.append(i)                                                    ## changes in list
print('Central European',ce) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()  
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in ce:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Central European']                                                          ## changes in column names
#df1
resto_df['Central European']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

Mediterranean=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Mediterranean':    ## changes in parameter
            Mediterranean.append(i)                                                    ## changes in list
print('Mediterranean',Mediterranean) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()  
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in Mediterranean:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Mediterranean']                                                          ## changes in column names
#df1
resto_df['Mediterranean']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

eu=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'European':    ## changes in parameter
            eu.append(i)                                                    ## changes in list
print('European',eu) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()  
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in eu:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['European']                                                          ## changes in column names
#df1
resto_df['European']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

vo=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Vegan Options':    ## changes in parameter
            vo.append(i)                                                    ## changes in list
print('Vegan Options',vo) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()  
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in vo:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Vegan Options']                                                          ## changes in column names
#df1
resto_df['Vegan Options']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

gfo=[]                                                                       ## changes in list
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Gluten Free Options':    ## changes in parameter
            gfo.append(i)                                                    ## changes in list
print('Gluten Free Options',gfo) # printing indexes                                                ## changes in list
    #print(len(response['results']['data'][i]['cuisine']))
    #print()  
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in gfo:                                                              ## changes in list
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Gluten Free Options']                                                          ## changes in column names
#df1
resto_df['Gluten Free Options']=df1                                                          ## changes in column names
#resto_df

#----------------------------------------------------------------------------------------------------------------------------

seafood=[]
no_of_Rows = len(response['results']['data'])
no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

for i in range(0,len(response['results']['data'])):
    #print(i)
    for j in range(0,len(response['results']['data'][i]['cuisine'])):
        #print(response['results']['data'][i]['cuisine'][j]['name'])
        if response['results']['data'][i]['cuisine'][j]['name'] == 'Seafood':
            seafood.append(i) 
print('Seafood',seafood) # printing indexes
    #print(len(response['results']['data'][i]['cuisine']))
    #print()  
no_of_Rows = len(response['results']['data'])

no_of_Cols = 1

df1 = pd.DataFrame(index=range(no_of_Rows),columns=range(no_of_Cols))

element = 'Available'
for ind in seafood:
    df1.iloc[ind]=element
df1=df1.fillna('N/A')
df1.columns=['Seafood']
#df1
resto_df['Seafood']=df1
#resto_df

Bar [10, 16, 20, 29, 32, 33, 33, 49]
Italian [1, 8, 9, 24, 27, 28, 38, 45]
Japanese [48]
American [6, 17, 37]
Steakhouse [25]
Cafe [26, 34, 35]
Indian [31]
Asian [11, 19, 29, 46, 48]
Healthy [1, 3, 27]
German [10, 14, 16, 18, 20, 32, 33, 36, 40, 42, 44, 47, 49]
Central European [10, 16, 18, 20, 25, 30, 32, 33, 36, 40, 43, 44, 45, 47, 49]
Mediterranean [0, 1, 2, 3, 4, 7, 8, 9, 15, 18, 24, 27, 28, 38, 45]
European [3, 5, 7, 8, 9, 10, 15, 16, 18, 20, 24, 25, 26, 28, 29, 30, 32, 33, 36, 40, 43, 44, 45, 47, 49]
Vegan Options [0, 1, 2, 3, 4, 5, 7, 9, 11, 12, 13, 15, 16, 17, 18, 19, 21, 26, 27, 29, 31, 33, 34, 38, 39, 41, 46, 48]
Gluten Free Options [0, 2, 3, 4, 5, 7, 9, 13, 14, 15, 18, 19, 20, 26, 27, 30, 38, 41, 46, 47, 48]
Seafood [2, 3, 4, 7, 8, 40]


In [27]:
resto_df.head(10)

,name,latitude,longitude,location_id,num_reviews,ranking_position,ranking_denominator,ranking_category,rating,address,cuisine,establishment_types,price,Bar,Italian,Japanese,American,Steakhouse,Cafe,Indian,Asian,Healthy,German,Central European,Mediterranean,European,Vegan Options,Gluten Free Options,Seafood
0,Restaurant Sonnenbad,49.012627,8.348418,5817948,213,1,604,restaurant,4.5,"Am Sonnenbad 1, 76189, Karlsruhe, Baden-Wurtte...",Mediterranean,Restaurants,$2 - $24,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,N/A,Available,Available,N/A
1,Pinsa Si,49.00722,8.39475,16810634,83,2,604,restaurant,5.0,"Karlstrasse 31, 76133 Karlsruhe, 76133, Karlsr...",Italian,Restaurants,None,N/A,Available,N/A,N/A,N/A,N/A,N/A,N/A,Available,N/A,N/A,Available,N/A,Available,N/A,N/A
2,Restaurant Terra Mare,49.003933,8.398199,7236591,423,3,604,restaurant,4.5,"Ritterstr. 19, 76137, Karlsruhe, Baden-Wurttem...",Seafood,Restaurants,$13 - $25,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,N/A,Available,Available,Available
3,Toro Tapasbar,49.01077,8.393829,4054709,577,4,604,restaurant,4.5,"Akademiestrasse 57, 76133, Karlsruhe, Baden-Wu...",Seafood,Restaurants,None,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,N/A,N/A,Available,Available,Available,Available,Available
4,Casa do Jose,49.00582,8.407025,3737103,337,5,604,restaurant,4.5,"Kriegsstr. 92, 76133, Karlsruhe, Baden-Wurttem...",Seafood,Restaurants,None,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,N/A,Available,Available,Available
5,Alte Brauerei,49.053356,8.531175,5531196,226,6,604,restaurant,4.5,"Bruchsaler Str. 22, 76356 Weingarten, Karlsruh...",Barbecue,Restaurants,None,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,Available,Available,N/A
6,DeliBurgers,49.01103,8.395794,9734212,276,7,604,restaurant,4.5,"Akademiestr. 39 Next to Europaplatz, 76133, Ka...",American,Restaurants,$11 - $20,N/A,N/A,N/A,Available,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
7,Sokrates,48.996128,8.391512,1040113,287,8,604,restaurant,4.5,"Welfenstr. 18, 76137, Karlsruhe, Baden-Wurttem...",Seafood,Restaurants,None,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,Available,Available,Available,Available
8,Il Teatro2,49.003975,8.40281,2073052,82,9,604,restaurant,5.0,"Ettlingerstrasse 2c, 76137, Karlsruhe, Baden-W...",Seafood,Restaurants,None,N/A,Available,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,Available,N/A,N/A,Available
9,Osteria Carre,49.00929,8.37632,6214144,180,10,604,restaurant,4.5,"Nelkenstr. 19, 76135, Karlsruhe, Baden-Wurttem...",Italian,Restaurants,None,N/A,Available,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,Available,Available,Available,N/A


* Making ranking categorical

In [28]:
resto_df['ranking_category'] = resto_df['ranking_position'].astype("int").apply(lambda x: 'A' if x <= 10 else ('B' if x>10 and x <= 20 else ('C' if x>20 and x <= 30 else ('D' if x>30 and x <= 40 else 'E'))))
#resto_df

* Splitting max and min price range

In [29]:
new = resto_df['price'].str.split('-',n=1,expand=True)
new = new.fillna(0)
resto_df['min price']=new[0]
resto_df['max price']=new[1]
#resto_df

*prices data for every restaurant is not available*

* Converting the rating category

In [30]:
resto_df['rating_category'] = resto_df['rating'].astype("float").apply(lambda x: 'A' if x > 4.5 else ('B' if x>4 and x<=4.5 else 'C'))
#resto_df

__Displaying the final dataframe__

In [31]:
pd.set_option('display.max_columns', None)
resto_df

,name,latitude,longitude,location_id,num_reviews,ranking_position,ranking_denominator,ranking_category,rating,address,cuisine,establishment_types,price,Bar,Italian,Japanese,American,Steakhouse,Cafe,Indian,Asian,Healthy,German,Central European,Mediterranean,European,Vegan Options,Gluten Free Options,Seafood,min price,max price,rating_category
0,Restaurant Sonnenbad,49.012627,8.348418,5817948,213,1,604,A,4.5,"Am Sonnenbad 1, 76189, Karlsruhe, Baden-Wurtte...",Mediterranean,Restaurants,$2 - $24,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,N/A,Available,Available,N/A,$2,$24,B
1,Pinsa Si,49.00722,8.39475,16810634,83,2,604,A,5.0,"Karlstrasse 31, 76133 Karlsruhe, 76133, Karlsr...",Italian,Restaurants,None,N/A,Available,N/A,N/A,N/A,N/A,N/A,N/A,Available,N/A,N/A,Available,N/A,Available,N/A,N/A,0,0,A
2,Restaurant Terra Mare,49.003933,8.398199,7236591,423,3,604,A,4.5,"Ritterstr. 19, 76137, Karlsruhe, Baden-Wurttem...",Seafood,Restaurants,$13 - $25,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,N/A,Available,Available,Available,$13,$25,B
3,Toro Tapasbar,49.01077,8.393829,4054709,577,4,604,A,4.5,"Akademiestrasse 57, 76133, Karlsruhe, Baden-Wu...",Seafood,Restaurants,None,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,N/A,N/A,Available,Available,Available,Available,Available,0,0,B
4,Casa do Jose,49.00582,8.407025,3737103,337,5,604,A,4.5,"Kriegsstr. 92, 76133, Karlsruhe, Baden-Wurttem...",Seafood,Restaurants,None,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,N/A,Available,Available,Available,0,0,B
5,Alte Brauerei,49.053356,8.531175,5531196,226,6,604,A,4.5,"Bruchsaler Str. 22, 76356 Weingarten, Karlsruh...",Barbecue,Restaurants,None,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,Available,Available,N/A,0,0,B
6,DeliBurgers,49.01103,8.395794,9734212,276,7,604,A,4.5,"Akademiestr. 39 Next to Europaplatz, 76133, Ka...",American,Restaurants,$11 - $20,N/A,N/A,N/A,Available,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,$11,$20,B
7,Sokrates,48.996128,8.391512,1040113,287,8,604,A,4.5,"Welfenstr. 18, 76137, Karlsruhe, Baden-Wurttem...",Seafood,Restaurants,None,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,Available,Available,Available,Available,0,0,B
8,Il Teatro2,49.003975,8.40281,2073052,82,9,604,A,5.0,"Ettlingerstrasse 2c, 76137, Karlsruhe, Baden-W...",Seafood,Restaurants,None,N/A,Available,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,Available,N/A,N/A,Available,0,0,A
9,Osteria Carre,49.00929,8.37632,6214144,180,10,604,A,4.5,"Nelkenstr. 19, 76135, Karlsruhe, Baden-Wurttem...",Italian,Restaurants,None,N/A,Available,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Available,Available,Available,Available,N/A,0,0,B


__Saving the dataframe as a CSV file__<br/>
*for future uses*

In [32]:
#resto_df.to_csv('Berlin_resto.csv', index = False)

*Repeating for all cities*

In [33]:
#resto_df.to_csv('Munich_resto.csv', index = False)
#resto_df.to_csv('Hamburg_resto.csv', index = False)
#resto_df.to_csv('Frankfurt_resto.csv', index = False)
#resto_df.to_csv('Essen_resto.csv', index = False)
#resto_df.to_csv('Stuttgart_resto.csv', index = False)
#resto_df.to_csv('Dortmund_resto.csv', index = False)
#resto_df.to_csv('Duesseldorf_resto.csv', index = False)
#resto_df.to_csv('Hannover_resto.csv', index = False)
#resto_df.to_csv('Leipzig_resto.csv', index = False)
#resto_df.to_csv('Duisburg_resto.csv', index = False)
#resto_df.to_csv('Dresden_resto.csv', index = False)
#resto_df.to_csv('Bochum_resto.csv', index = False)
#resto_df.to_csv('Wuppertal_resto.csv', index = False)
#resto_df.to_csv('Bielefeld_resto.csv', index = False)
#resto_df.to_csv('Bonn_resto.csv', index = False)
#resto_df.to_csv('Mannheim_resto.csv', index = False)
#resto_df.to_csv('Cologne_resto.csv', index = False)
#resto_df.to_csv('Nuremberg_resto.csv', index = False)
resto_df.to_csv('Karlsruhe_resto.csv', index = False)